In [1]:
from bokeh.io import show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Label, Title, HoverTool, Text
from bokeh.plotting import figure
from bokeh.util.compiler import TypeScript

import numpy as np
import pandas as pd

from scipy.signal import welch

output_notebook()

Loading BokehJS ...

In [2]:
from bokeh.models.annotations import Span

In [3]:
no_of_samples = 256

In [4]:
# Sampling frequency will be one hour
time_index = pd.date_range('2018-01-01', periods=no_of_samples, freq='H')

In [5]:
freq = 0.15
t = np.arange(no_of_samples)
sinusoid = np.sin(2*np.pi*t*freq)
data = np.random.random(size=(3, no_of_samples))

noisy_sinusoids = [sinusoid + (0.5) * np.random.random(no_of_samples) for x in range(3)]
s1, s2, s3 = noisy_sinusoids

# Detided
s2 = s2 - ((0.2) * np.random.random(no_of_samples) + sinusoid)
s3 = s3 - ((0.1) * np.random.random(no_of_samples) + sinusoid)

In [6]:
sampling_freq = 1 #hour
freqs = list(np.linspace(0, sampling_freq * 0.5, no_of_samples // 2 + 1))
fft1, fft2, fft3 = map(lambda datum: np.abs(np.fft.rfft(datum, norm="ortho")), noisy_sinusoids)

In [7]:
ts1 = pd.Series(s1, index=time_index).to_frame()
ts2 = pd.Series(s2, index=time_index).to_frame()
ts3 = pd.Series(s3, index=time_index).to_frame()

In [8]:
fs1 = pd.Series(fft1, index=freqs).to_frame()
fs2 = pd.Series(fft2, index=freqs).to_frame()
fs3 = pd.Series(fft3, index=freqs).to_frame()

In [9]:
src1 = ColumnDataSource({"time": time_index, "vals": s1})

In [10]:
src2 = ColumnDataSource({"time": time_index, "vals": s2})
src3 = ColumnDataSource({"time": time_index, "vals": s3})

In [11]:
fftsrc1 = ColumnDataSource({"freq": freqs, "vals": fft1})
fftsrc2 = ColumnDataSource({"freq": freqs, "vals": fft2})
fftsrc3 = ColumnDataSource({"freq": freqs, "vals": fft3})

In [12]:
p_lone = figure(plot_width=800, plot_height=350, title="Raw observational data", x_axis_type="datetime")
p_lone.line(x='time', y='vals', line_color='navy', line_dash="solid", legend_label="raw_obs", source=src1)

p_lone.xaxis.axis_label = "Date"
p_lone.yaxis.axis_label = "Water level (m)"

show(p_lone)

In [13]:
twl_hovertool = HoverTool(
    tooltips=[
        ('date', '@time{%F}'),
        ('wlev', '@vals'),
    ],
    
    formatters={
        '@time' : 'datetime'
    }
)

p_lone.add_tools(twl_hovertool)

In [14]:
p_double = figure(plot_width=800, plot_height = 350,
                  x_axis_type="datetime", 
                  title="Detided observation vs detided model")

p_double.line(x="time", y="vals", line_color="navy", line_dash="solid", legend_label="detided_obs", 
              muted_color="navy", muted_alpha=0.2, source=src2)

p_double.line(x="time", y="vals", line_color="maroon", line_dash="solid", legend_label="detided_mod",
              muted_color="maroon", muted_alpha=0.2, source=src3)

p_double.xaxis.axis_label = "Date"
p_double.yaxis.axis_label = "Water level (m)"

p_double.legend.click_policy="mute"

show(p_double)

In [34]:
p_fft = figure(plot_width=800, plot_height=350, title="Discrete fourier transform of observation/model outputs",
               y_axis_type="log")

p_fft.line(x="freq", y="vals", line_color="navy", line_dash="solid", legend_label="raw_obs",
           muted_color="navy", muted_alpha=0.2, source=fftsrc1)

p_fft.line(x="freq", y="vals", line_color="maroon", line_dash="solid", legend_label="detided_obs",
           muted_color="maroon", muted_alpha=0.2, source=fftsrc2)

p_fft.line(x="freq", y="vals", line_color="forestgreen", legend_label="detided_mod",
           muted_color="forestgreen", muted_alpha=0.2, source=fftsrc3)

p_fft.ray(x=[0], y=[0], length=0, angle=(np.pi)/2, line_color="hotpink",
          line_dash="dashed", legend_label="48H")
p_fft.ray(x=[0], y=[0], length=0, angle=(np.pi)/2, line_color="springgreen",
          line_dash="dashed", legend_label="24H")
p_fft.ray(x=[0], y=[0], length=0, angle=(np.pi)/2, line_color="darkviolet",
          line_dash="dashed", legend_label="12H")

p_fft.add_layout(Span(dimension="height", location=1/48, line_color="hotpink", line_dash="dashed"))
p_fft.add_layout(Span(dimension="height", location=1/24, line_color="springgreen", line_dash="dashed"))
p_fft.add_layout(Span(dimension="height", location=1/12, line_color="darkviolet", line_dash="dashed"))

text_coords = ColumnDataSource(dict(x=[0.48, 0.48], y=[1.3, 1], text=[f"\u03C3 = 1", f"\u03B3\u00B2 = 2"]))
glph = Text(x='x', y='y', text="text", text_font_size="10pt")
p_fft.add_glyph(text_coords, glph)

p_fft.xaxis.axis_label = "Frequency (h^-1)"
p_fft.yaxis.axis_label = ""

p_fft.legend.click_policy="mute"

'''
low_label = Label(x=700, y=70,
                  x_units="screen", y_units="screen",
                  text=f"""foo{1}\nbar""", 
                  background_fill_alpha=0.7, border_line_color='black')

p_fft.add_layout(low_label)
'''

show(p_fft)

In [16]:
### Interactions ####
# Linked panning
p_double.x_range = p_lone.x_range
p_double.y_range = p_lone.y_range

In [17]:
column_layout = column(p_lone, p_double, p_fft)
show(column_layout)